<font color=red>This lecture is a work in progress that will be finalized on 21 June 2022.</font>

# Running Marlowe Contacts on the Blockchain

In this lecture we demonstrate how to use `marlowe-cli` to run Marlowe contracts on the blockchain, but without using the Plutus Application Backend (PAB). We do this for the escrow contract of the previous lecture.

## Contract Parameters

In the previous lecture, we ran the contract without submitting transactions. There we set the following parameters.

In [7]:
INITIAL_LOVELACE=3000000             # The creation transaction will deposit 3₳.
PRICE=256000000                      # The prices of the item is 256₳.
MINIMUM_ADA=2000000                  # The minimum lovelace to be included with native token ouptut.

MEDIATOR_ROLE=CM                     # Christopher Marlowe is the mediator.
BUYER_ROLE=TM                        # Thomas Middleton is the buyer.
SELLER_ROLE=FB                       # Francis Beaumont (FB) is the seller.

NOW=$(($(date -u +%s)*1000))         # The current time in POSIX milliseconds.
HOUR=$((60*60*1000))                 # One hour, in POSIX milliseconds.
PAYMENT_DEADLINE=$((NOW+10*HOUR))    # The payment deadline, ten hours from now.
COMPLAINT_DEADLINE=$((NOW+12*HOUR))  # The complaint deadline, twelve hours from now.
DISPUTE_DEADLINE=$((NOW+14*HOUR))    # The dispute deadline, fourteen hours from now.
MEDIATION_DEADLINE=$((NOW+16*HOUR))  # The mediation deadline, sixteen hours from now.

ROLES_CURRENCY=38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa

MEDIATOR_TOKEN="$ROLES_CURRENCY.$MEDIATOR_ROLE"
BUYER_TOKEN="$ROLES_CURRENCY.$BUYER_ROLE"
SELLER_TOKEN="$ROLES_CURRENCY.$SELLER_ROLE"

In [2]:
WALLET_SKEY=my-wallet.skey
WALLET_ADDRESS=$(cat my-wallet.address)

In [3]:
export CARDANO_NODE_SOCKET_PATH=~/.local/share/Daedalus/marlowe_pioneers/cardano-node.socket

## Creating Signing Keys for Roles

The escrow contract involves three participants (Mediator, Buyer, Seller), so a real-life scenario for it involves three wallets.
*   The next sections shows to create signing keys and addresses for these roles.
*   Alternatively, one can create the signing keys for three wallets using the instructions in the lecture on installing Marlowe tools.

### Keys and Address for the Mediator

First set variables related to the mediator.

In [4]:
MEDIATOR_ROLE="$MEDIATOR_ROLE"
MEDIATOR_SKEY=mediator.skey
MEDIATOR_VKEY=mediator.vkey

Generate keys for the mediator and compute their address.

In [3]:
cardano-cli address key-gen --signing-key-file "$MEDIATOR_SKEY" --verification-key-file "$MEDIATOR_VKEY"
MEDIATOR_ADDRESS=$(cardano-cli address build --testnet-magic 1567 --payment-verification-key-file "$MEDIATOR_VKEY")
echo $MEDIATOR_ADDRESS

addr_test1vqn27wu5zrhnc2gw7vvsflspk43s205eq8anumr2f8tmy4qced6qj


### Keys and Address for the Buyer and Seller

In [4]:
BUYER_ROLE="$BUYER_ROLE"
BUYER_SKEY=buyer.skey
BUYER_VKEY=buyer.vkey
cardano-cli address key-gen --signing-key-file "$BUYER_SKEY" --verification-key-file "$BUYER_VKEY"
BUYER_ADDRESS=$(cardano-cli address build --testnet-magic 1567 --payment-verification-key-file "$BUYER_VKEY")
echo $BUYER_ADDRESS

addr_test1vpj7f4m5mpwv4k9yatzk3pynwta3dmy8c5gk4tn2l9lxdxcnun0wm


In [5]:
SELLER_ROLE="$SELLER_ROLE"
SELLER_SKEY=seller.skey
SELLER_VKEY=seller.vkey
cardano-cli address key-gen --signing-key-file "$SELLER_SKEY" --verification-key-file "$SELLER_VKEY"
SELLER_ADDRESS=$(cardano-cli address build --testnet-magic 1567 --payment-verification-key-file "$SELLER_VKEY")
echo $SELLER_ADDRESS

addr_test1vr8ngyhpm4k8hytahpl9c08c0pj6de7ddys0zxgex2z842swlujyf


### Distribute Role Tokens and Ada

In the previous lecture we minted role tokens for this contract. They are in UTxOs held at the address in `WALLET_ADDRESS=my-wallet.address`.

In [9]:
cardano-cli query utxo --testnet-magic 1567 --address "$WALLET_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f2728cdf78b21b9da70851a16f340a94d64302957eb76a09f8a821a7e2769a7     0        969821035 lovelace + TxOutDatumNone
7f2728cdf78b21b9da70851a16f340a94d64302957eb76a09f8a821a7e2769a7     1        10000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.434d + TxOutDatumNone
7f2728cdf78b21b9da70851a16f340a94d64302957eb76a09f8a821a7e2769a7     2        10000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.4642 + TxOutDatumNone
7f2728cdf78b21b9da70851a16f340a94d64302957eb76a09f8a821a7e2769a7     3        10000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.544d + TxOutDatumNone


Here is the correspondence between the role names and their hexadecimal representations.

Now distribute the tokens and some ADA, 20 ADA to the mediator, 300 ADA to the buyer, and 40 ADA to the seller.

In [13]:
TX_IN=7f2728cdf78b21b9da70851a16f340a94d64302957eb76a09f8a821a7e2769a7

marlowe-cli transaction simple --testnet-magic 1567                                                   \
                               --socket-path "$CARDANO_NODE_SOCKET_PATH"                              \
                               --required-signer "$WALLET_SKEY"                                       \
                               --tx-in "$TX_IN#0"                                                     \
                               --tx-in "$TX_IN#1"                                                     \
                               --tx-in "$TX_IN#2"                                                     \
                               --tx-in "$TX_IN#3"                                                     \
                               --change-address "$WALLET_ADDRESS"                                     \
                               --tx-out "$MEDIATOR_ADDRESS+20000000+1 $ROLES_CURRENCY.$MEDIATOR_ROLE" \
                               --tx-out "$BUYER_ADDRESS+300000000+1 $ROLES_CURRENCY.$BUYER_ROLE"      \
                               --tx-out "$SELLER_ADDRESS+40000000+1 $ROLES_CURRENCY.$SELLER_ROLE"     \
                               --out-file /dev/null                                                   \
                               --submit 600

TxId "7946381c2ae077c460df3377387743a76c6a82cc16b20a56323a3b6c38feb4e4"


### Ensure That Each Address Has a Tokenless UTxO for Collateral

The command `marlowe-cli util clean` reorganizes UTxOs at an address so that there is a tokenless UTxO that is suitable for use as collateral.

In [18]:
marlowe-cli util clean --testnet-magic 1567                       \
                       --socket-path "$CARDANO_NODE_SOCKET_PATH"  \
                       --required-signer "$MEDIATOR_SKEY"         \
                       --change-address "$MEDIATOR_ADDRESS"       \
                       --out-file /dev/null                       \
                       --submit=600

TxId "ed7e0f9ff1dfe30766bd1a79f098740fed80e704d60b8887fb15b23b0f5f1521"


In [19]:
marlowe-cli util clean --testnet-magic 1567                      \
                       --socket-path "$CARDANO_NODE_SOCKET_PATH" \
                       --required-signer "$BUYER_SKEY"           \
                       --change-address "$BUYER_ADDRESS"         \
                       --out-file /dev/null                      \
                       --submit=600

TxId "a51decbc027a28060b9cee0a1538cccfa106e076a42f399ba77def6d61f97e09"


In [20]:
marlowe-cli util clean --testnet-magic 1567                      \
                       --socket-path "$CARDANO_NODE_SOCKET_PATH" \
                       --required-signer "$SELLER_SKEY"          \
                       --change-address "$SELLER_ADDRESS"        \
                       --out-file /dev/null                      \
                       --submit=600

TxId "fa641af74ac6744d02b6ba3e39bc40d4edb6faee17562049776b5a896d57922f"


### Check That Funds Have Been Distributed Correctly

The `cardano-cli` tool outputs token names in hexadecimal instead of ASCII. Here is the correspondence between the role names and their hexadecimal representations.

In [21]:
for n in "$MEDIATOR_ROLE" "$BUYER_ROLE" "$SELLER_ROLE"
do
  echo "$n=$(echo -n $n | basenc --base16 | tr '[:upper:]' '[:lower:]')"
done

CM=434d
TM=544d
FB=4642


In [22]:
cardano-cli query utxo --testnet-magic 1567 --address "$MEDIATOR_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ed7e0f9ff1dfe30766bd1a79f098740fed80e704d60b8887fb15b23b0f5f1521     0        17831023 lovelace + TxOutDatumNone
ed7e0f9ff1dfe30766bd1a79f098740fed80e704d60b8887fb15b23b0f5f1521     1        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.434d + TxOutDatumNone


In [23]:
cardano-cli query utxo --testnet-magic 1567 --address "$BUYER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a51decbc027a28060b9cee0a1538cccfa106e076a42f399ba77def6d61f97e09     0        297831023 lovelace + TxOutDatumNone
a51decbc027a28060b9cee0a1538cccfa106e076a42f399ba77def6d61f97e09     1        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.544d + TxOutDatumNone


In [24]:
cardano-cli query utxo --testnet-magic 1567 --address "$SELLER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
fa641af74ac6744d02b6ba3e39bc40d4edb6faee17562049776b5a896d57922f     0        37831023 lovelace + TxOutDatumNone
fa641af74ac6744d02b6ba3e39bc40d4edb6faee17562049776b5a896d57922f     1        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.4642 + TxOutDatumNone


In [1]:
TX_0_MEDIATOR_ADA=ed7e0f9ff1dfe30766bd1a79f098740fed80e704d60b8887fb15b23b0f5f1521#0
TX_0_MEDIATOR_TOKEN=ed7e0f9ff1dfe30766bd1a79f098740fed80e704d60b8887fb15b23b0f5f1521#1

TX_0_BUYER_ADA=a51decbc027a28060b9cee0a1538cccfa106e076a42f399ba77def6d61f97e09#0
TX_0_BUYER_TOKEN=a51decbc027a28060b9cee0a1538cccfa106e076a42f399ba77def6d61f97e09#1

TX_0_SELLER_ADA=fa641af74ac6744d02b6ba3e39bc40d4edb6faee17562049776b5a896d57922f#0
TX_0_SELLER_TOKEN=fa641af74ac6744d02b6ba3e39bc40d4edb6faee17562049776b5a896d57922f#1

## Create the Contract and Its Initial State

In [25]:
marlowe-cli template escrow --minimum-ada "$INITIAL_LOVELACE"          \
                            --price "$PRICE"                           \
                            --seller "Role=$SELLER_ROLE"               \
                            --buyer "Role=$BUYER_ROLE"                 \
                            --mediator "Role=$MEDIATOR_ROLE"           \
                            --payment-deadline "$PAYMENT_DEADLINE"     \
                            --complaint-deadline "$COMPLAINT_DEADLINE" \
                            --dispute-deadline "$DISPUTE_DEADLINE"     \
                            --mediation-deadline "$MEDIATION_DEADLINE" \
                            --out-contract-file tx-1.contract          \
                            --out-state-file    tx-1.state

## Transaction 1: Mediator Creates Escrow Contract with Initial ADA

First create the `.marlowe` file that containts the contract, its state, Plutus data, and network information.

In [26]:
marlowe-cli run initialize --testnet-magic 1567                      \
                           --socket-path "$CARDANO_NODE_SOCKET_PATH" \
                           --roles-currency "$ROLES_CURRENCY"        \
                           --contract-file tx-1.contract             \
                           --state-file    tx-1.state                \
                           --out-file      tx-1.marlowe              \
                           --print-stats


Validator size: 12633
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 24920101, exBudgetMemory = ExMemory 83800}


The mediator uses their tokenless UTxO to fund the contract.

In [31]:
marlowe-cli run execute --testnet-magic 1567                      \
                        --socket-path "$CARDANO_NODE_SOCKET_PATH" \
                        --tx-in "$TX_0_MEDIATOR_ADA"              \
                        --change-address "$MEDIATOR_ADDRESS"      \
                        --required-signer "$MEDIATOR_SKEY"        \
                        --marlowe-out-file tx-1.marlowe           \
                        --out-file tx-1.raw                       \
                        --print-stats                             \
                        --submit=600


Fee: Lovelace 197709
Size: 719 / 32768 = 2%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%
TxId "bc4b7b7979da6f9f7f01b85ce3b44eacac838547944ef813a4059e19c75985be"


In [6]:
TX_1=bc4b7b7979da6f9f7f01b85ce3b44eacac838547944ef813a4059e19c75985be

## Results of Transaction 1

The mediator now has a little less ADA because they deposited 3 ADA.

In [32]:
cardano-cli query utxo --testnet-magic 1567 --address "$MEDIATOR_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
bc4b7b7979da6f9f7f01b85ce3b44eacac838547944ef813a4059e19c75985be     0        14633314 lovelace + TxOutDatumNone
ed7e0f9ff1dfe30766bd1a79f098740fed80e704d60b8887fb15b23b0f5f1521     1        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.434d + TxOutDatumNone


The Plutus script for the Marlowe contract now contains the 3 ADA that the mediator deposited.

In [9]:
CONTRACT_ADDRESS=$(jq -r '.marloweValidator.address' tx-1.marlowe)
echo "$CONTRACT_ADDRESS"

addr_test1wrcnw2f52qm6nytgakq25sy06kp9l4p9ne2wwn0smg4qr6getwzsc


In [34]:
cardano-cli query utxo --testnet-magic 1567 --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
bc4b7b7979da6f9f7f01b85ce3b44eacac838547944ef813a4059e19c75985be     1        3000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "33a0df53ce247af0cc7ff8f7655d56a5e7d3bb685cf060ff7a79d8f978782bca"


## Transaction 2: Buyer Deposits Funds into Seller’s Account

In [35]:
marlowe-cli run prepare --marlowe-file tx-1.marlowe           \
                        --deposit-account "Role=$SELLER_ROLE" \
                        --deposit-party "Role=$BUYER_ROLE"    \
                        --deposit-amount "$PRICE"             \
                        --invalid-before "$NOW"               \
                        --invalid-hereafter "$((NOW+9*HOUR))" \
                        --out-file tx-2.marlowe               \
                        --print-stats


Datum size: 463


The buyer uses their UTxOs and the script UTxO to deposit their funds.

In [39]:
marlowe-cli run execute --testnet-magic 1567                                 \
                        --socket-path "$CARDANO_NODE_SOCKET_PATH"             \
                        --marlowe-in-file tx-1.marlowe                        \
                        --tx-in-marlowe "$TX_1"#1                             \
                        --tx-in-collateral "$TX_0_BUYER_ADA"                  \
                        --tx-in "$TX_0_BUYER_ADA"                             \
                        --tx-in "$TX_0_BUYER_TOKEN"                           \
                        --required-signer "$BUYER_SKEY"                       \
                        --marlowe-out-file tx-2.marlowe                       \
                        --tx-out "$BUYER_ADDRESS+$MINIMUM_ADA+1 $BUYER_TOKEN" \
                        --change-address "$BUYER_ADDRESS"                     \
                        --out-file tx-2.raw                                   \
                        --print-stats                                         \
                        --submit=600


Fee: Lovelace 1292781
Size: 14063 / 32768 = 42%
Execution units:
  Memory: 5989056 / 30000000 = 19%
  Steps: 2127473978 / 10000000000 = 21%
TxId "fd7dcb45cc607c21133087112f65ab5fbdb62e2d4359205aee2cb0f1ef2626b6"


In [13]:
TX_2=fd7dcb45cc607c21133087112f65ab5fbdb62e2d4359205aee2cb0f1ef2626b6

## Results of Transaction 2

The buyer now has a less ADA because they deposited 256 ADA.

In [7]:
cardano-cli query utxo --testnet-magic 1567 --address "$BUYER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
fd7dcb45cc607c21133087112f65ab5fbdb62e2d4359205aee2cb0f1ef2626b6     0        40538242 lovelace + TxOutDatumNone
fd7dcb45cc607c21133087112f65ab5fbdb62e2d4359205aee2cb0f1ef2626b6     2        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.544d + TxOutDatumNone


The Plutus script for the Marlowe contract now contains the additional 256 ADA that the buyer deposited.

In [10]:
cardano-cli query utxo --testnet-magic 1567 --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
fd7dcb45cc607c21133087112f65ab5fbdb62e2d4359205aee2cb0f1ef2626b6     1        259000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "43cce5b779782b0ae81c3641ecc6a0cb53079a842d0fa70fdca458a9a061d408"


## Transaction 3: The Buyer Reports That There is a Problem

In [11]:
marlowe-cli run prepare --marlowe-file tx-2.marlowe           \
                        --choice-name "Report problem"        \
                        --choice-party "Role=$BUYER_ROLE"     \
                        --choice-number 1                     \
                        --invalid-before "$NOW"               \
                        --invalid-hereafter "$((NOW+9*HOUR))" \
                        --out-file tx-3.marlowe               \
                        --print-stats


Datum size: 341
Payment 1
  Acccount: "FB"
  Payee: Account "TM"
  Ada: 256.000000


The buyer uses their UTxOs and the script UTxO to report that there is a problem.

In [14]:
marlowe-cli run execute --testnet-magic 1567                                  \
                        --socket-path "$CARDANO_NODE_SOCKET_PATH"             \
                        --marlowe-in-file tx-2.marlowe                        \
                        --tx-in-marlowe "$TX_2"#1                             \
                        --tx-in-collateral "$TX_2"#0                          \
                        --tx-in "$TX_2"#0                                     \
                        --tx-in "$TX_2"#2                                     \
                        --required-signer "$BUYER_SKEY"                       \
                        --marlowe-out-file tx-3.marlowe                       \
                        --tx-out "$BUYER_ADDRESS+$MINIMUM_ADA+1 $BUYER_TOKEN" \
                        --change-address "$BUYER_ADDRESS"                     \
                        --out-file tx-3.raw                                   \
                        --print-stats                                         \
                        --submit=600


Fee: Lovelace 1372588
Size: 13918 / 32768 = 42%
Execution units:
  Memory: 7094094 / 30000000 = 23%
  Steps: 2438518154 / 10000000000 = 24%
TxId "4c4462d48a2e4026944e65b8cfa61875039d4f6824b6239a76528a45194a52b6"


In [15]:
TX_3=4c4462d48a2e4026944e65b8cfa61875039d4f6824b6239a76528a45194a52b6

## Results of Transaction 3

The buyer now has a approximately the same amount of ADA because they only reported the problem and didn't deposit anything.

In [16]:
cardano-cli query utxo --testnet-magic 1567 --address "$BUYER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
4c4462d48a2e4026944e65b8cfa61875039d4f6824b6239a76528a45194a52b6     0        39165654 lovelace + TxOutDatumNone
4c4462d48a2e4026944e65b8cfa61875039d4f6824b6239a76528a45194a52b6     2        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.544d + TxOutDatumNone


The ADA at the Plutus script hasn't changed, but the datum there has changed.

In [17]:
cardano-cli query utxo --testnet-magic 1567 --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
4c4462d48a2e4026944e65b8cfa61875039d4f6824b6239a76528a45194a52b6     1        259000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "9c49d9dc8032128313324b6886951114de727d493c17607ffc1053a94467772e"


## Transaction 4: The Seller Disputes that There is a Problem

In [29]:
marlowe-cli run prepare --marlowe-file tx-3.marlowe            \
                        --choice-name "Dispute problem"        \
                        --choice-party "Role=$SELLER_ROLE"     \
                        --choice-number 0                      \
                        --invalid-before "$NOW"                \
                        --invalid-hereafter "$((NOW+9*HOUR))"  \
                        --out-file tx-4.marlowe                \
                        --print-stats


Datum size: 262


The seller uses their UTxOs and the script UTxO to dispute that there is a problem.

In [30]:
marlowe-cli run execute --testnet-magic 1567                                   \
                        --socket-path "$CARDANO_NODE_SOCKET_PATH"               \
                        --marlowe-in-file tx-3.marlowe                          \
                        --tx-in-marlowe "$TX_3"#1                               \
                        --tx-in-collateral "$TX_0_SELLER_ADA"                   \
                        --tx-in "$TX_0_SELLER_ADA"                              \
                        --tx-in "$TX_0_SELLER_TOKEN"                            \
                        --required-signer "$SELLER_SKEY"                        \
                        --marlowe-out-file tx-4.marlowe                         \
                        --tx-out "$SELLER_ADDRESS+$MINIMUM_ADA+1 $SELLER_TOKEN" \
                        --change-address "$SELLER_ADDRESS"                      \
                        --out-file tx-4.raw                                     \
                        --print-stats                                           \
                        --submit=600


Fee: Lovelace 1249155
Size: 13718 / 32768 = 41%
Execution units:
  Memory: 5703168 / 30000000 = 19%
  Steps: 1961727732 / 10000000000 = 19%
TxId "53d91ef7534151278fecabb2f733c241ffbee0b488aa0dba4d9376a151170c7e"


In [31]:
TX_4=53d91ef7534151278fecabb2f733c241ffbee0b488aa0dba4d9376a151170c7e

## Results of Transaction 4

The seller now has a approximately the same amount of ADA because they only disputed the problem and didn't deposit anything.

In [32]:
cardano-cli query utxo --testnet-magic 1567 --address "$SELLER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
53d91ef7534151278fecabb2f733c241ffbee0b488aa0dba4d9376a151170c7e     0        36581868 lovelace + TxOutDatumNone
53d91ef7534151278fecabb2f733c241ffbee0b488aa0dba4d9376a151170c7e     2        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.4642 + TxOutDatumNone


The ADA at the Plutus script hasn't changed, but the datum there has changed.

In [33]:
cardano-cli query utxo --testnet-magic 1567 --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
53d91ef7534151278fecabb2f733c241ffbee0b488aa0dba4d9376a151170c7e     1        259000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "6756b91d04aa84ca78cac9f88356c4636e5c1df68ea31c7e56a5dc5c758d83b0"


## Transaction 5: The Mediator Dismisses the Claim

In [34]:
marlowe-cli run prepare --marlowe-file tx-4.marlowe           \
                        --choice-name "Dismiss claim"         \
                        --choice-party "Role=$MEDIATOR_ROLE"  \
                        --choice-number 0                     \
                        --invalid-before "$NOW"               \
                        --invalid-hereafter "$((NOW+9*HOUR))" \
                        --out-file tx-5.marlowe               \
                        --print-stats


Datum size: 104
Payment 1
  Acccount: "TM"
  Payee: Account "FB"
  Ada: 256.000000
Payment 2
  Acccount: "CM"
  Payee: Party "CM"
  Ada: 3.000000
Payment 3
  Acccount: "FB"
  Payee: Party "FB"
  Ada: 256.000000


The mediator user their UTxOs and the script UTxO to dismiss the claim, in favor of the seller.

In [36]:
marlowe-cli run execute --testnet-magic 1567                                        \
                        --socket-path "$CARDANO_NODE_SOCKET_PATH"                   \
                        --marlowe-in-file tx-4.marlowe                              \
                        --tx-in-marlowe "$TX_4"#1                                   \
                        --tx-in-collateral "$TX_1"#0                                \
                        --tx-in "$TX_1"#0                                           \
                        --tx-in "$TX_0_MEDIATOR_TOKEN"                              \
                        --required-signer "$MEDIATOR_SKEY"                          \
                        --marlowe-out-file tx-5.marlowe                             \
                        --tx-out "$MEDIATOR_ADDRESS+$MINIMUM_ADA+1 $MEDIATOR_TOKEN" \
                        --change-address "$MEDIATOR_ADDRESS"                        \
                        --out-file tx-5.raw                                         \
                        --print-stats                                               \
                        --submit=600


Fee: Lovelace 1239526
Size: 13452 / 32768 = 41%
Execution units:
  Memory: 5759762 / 30000000 = 19%
  Steps: 1945217056 / 10000000000 = 19%
TxId "4f597e8a4fcdc36da5326e82989005be03e1c0b4f7f0b30baa0cf31fb266c902"


In [37]:
TX_5=4f597e8a4fcdc36da5326e82989005be03e1c0b4f7f0b30baa0cf31fb266c902

## Results of Transaction 5

The mediator now has a approximately the same amount of ADA because they only settled the claim and didn't deposit anything.

In [38]:
cardano-cli query utxo --testnet-magic 1567 --address "$MEDIATOR_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
4f597e8a4fcdc36da5326e82989005be03e1c0b4f7f0b30baa0cf31fb266c902     0        13393788 lovelace + TxOutDatumNone
4f597e8a4fcdc36da5326e82989005be03e1c0b4f7f0b30baa0cf31fb266c902     3        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.434d + TxOutDatumNone


The contract is complete, so there is no ADA at the Plutus script address.

In [39]:
cardano-cli query utxo --testnet-magic 1567 --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The payout script address now contains payments that the seller and mediator can claim.

In [40]:
ROLE_ADDRESS=$(jq -r '.rolesValidator.address' tx-1.marlowe)
echo "$ROLE_ADDRESS"

addr_test1wphwyse50dp4ev7uettewcurynu00c7frs8acrs9avpvcugd93us4


In [41]:
cardano-cli query utxo --testnet-magic 1567 --address "$ROLE_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
4f597e8a4fcdc36da5326e82989005be03e1c0b4f7f0b30baa0cf31fb266c902     1        3000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "1d8ad8b0a642297fa38291e4522015f82d47774e9a9829345a0952c654f427a3"
4f597e8a4fcdc36da5326e82989005be03e1c0b4f7f0b30baa0cf31fb266c902     2        256000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "f605a13bb90b0f530748173887182b9135318402c3d5c989e40cca12b283664b"


## Transaction 6: The Seller Withdraws Their Funds

The `marlowe-cli run withdraw` command lets the seller withdraw their funds from the payout script address.

In [42]:
marlowe-cli run withdraw --testnet-magic 1567                                    \
                         --socket-path "$CARDANO_NODE_SOCKET_PATH"               \
                         --marlowe-file tx-5.marlowe                             \
                         --role-name "$SELLER_ROLE"                              \
                         --tx-in "$TX_4"#0                                       \
                         --tx-in "$TX_4"#2                                       \
                         --tx-in-collateral "$TX_4"#0                            \
                         --required-signer "$SELLER_SKEY"                        \
                         --tx-out "$SELLER_ADDRESS+$MINIMUM_ADA+1 $SELLER_TOKEN" \
                         --change-address "$SELLER_ADDRESS"                      \
                         --out-file tx-6.raw                                     \
                         --print-stats                                           \
                         --submit=600


Fee: Lovelace 426563
Size: 2885 / 32768 = 8%
Execution units:
  Memory: 1461810 / 30000000 = 4%
  Steps: 557930172 / 10000000000 = 5%
TxId "7ba827a37c5b4d59d8d8e5571089a169d8d5662f6b3a05e83c52c3e6852de1e5"


In [43]:
TX_6=7ba827a37c5b4d59d8d8e5571089a169d8d5662f6b3a05e83c52c3e6852de1e5

## Transaction 7: The Mediator Withdraws Their Funds

In [44]:
marlowe-cli run withdraw --testnet-magic 1567                                        \
                         --socket-path "$CARDANO_NODE_SOCKET_PATH"                   \
                         --marlowe-file tx-5.marlowe                                 \
                         --role-name "$MEDIATOR_ROLE"                                \
                         --tx-in "$TX_5"#0                                           \
                         --tx-in "$TX_5"#3                                           \
                         --tx-in-collateral "$TX_5"#0                                \
                         --required-signer "$MEDIATOR_SKEY"                          \
                         --tx-out "$MEDIATOR_ADDRESS+$MINIMUM_ADA+1 $MEDIATOR_TOKEN" \
                         --change-address "$MEDIATOR_ADDRESS"                        \
                         --out-file tx-7.raw                                         \
                         --print-stats                                               \
                         --submit=600


Fee: Lovelace 426563
Size: 2885 / 32768 = 8%
Execution units:
  Memory: 1461810 / 30000000 = 4%
  Steps: 557930172 / 10000000000 = 5%
TxId "86008a0ab907a413ab8b749c919f43cfbb36e605f165b3c7a55e1744b789c1ba"


In [45]:
TX_7=86008a0ab907a413ab8b749c919f43cfbb36e605f165b3c7a55e1744b789c1ba

## Results of Transactions 6 and 7

The payout script address has no funds.

In [46]:
cardano-cli query utxo --testnet-magic 1567 --address "$ROLE_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The mediator and seller have the funds they withdrew.

In [47]:
cardano-cli query utxo --testnet-magic 1567 --address "$MEDIATOR_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
86008a0ab907a413ab8b749c919f43cfbb36e605f165b3c7a55e1744b789c1ba     0        12967225 lovelace + TxOutDatumNone
86008a0ab907a413ab8b749c919f43cfbb36e605f165b3c7a55e1744b789c1ba     1        3000000 lovelace + TxOutDatumNone
86008a0ab907a413ab8b749c919f43cfbb36e605f165b3c7a55e1744b789c1ba     2        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.434d + TxOutDatumNone


In [48]:
cardano-cli query utxo --testnet-magic 1567 --address "$SELLER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7ba827a37c5b4d59d8d8e5571089a169d8d5662f6b3a05e83c52c3e6852de1e5     0        36155305 lovelace + TxOutDatumNone
7ba827a37c5b4d59d8d8e5571089a169d8d5662f6b3a05e83c52c3e6852de1e5     1        256000000 lovelace + TxOutDatumNone
7ba827a37c5b4d59d8d8e5571089a169d8d5662f6b3a05e83c52c3e6852de1e5     2        2000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.4642 + TxOutDatumNone


## Clean Up

We send the funds and role tokens back to the wallet.

In [52]:
marlowe-cli transaction simple --testnet-magic 1567                                                \
                               --socket-path "$CARDANO_NODE_SOCKET_PATH"                           \
                               --required-signer "$MEDIATOR_SKEY"                                  \
                               --tx-in "$TX_7#0" --tx-in "$TX_7#1" --tx-in "$TX_7#2"               \
                               --required-signer "$BUYER_SKEY"                                     \
                               --tx-in "$TX_3#0" --tx-in "$TX_3#2"                                 \
                               --required-signer "$SELLER_SKEY"                                    \
                               --tx-in "$TX_6#0" --tx-in "$TX_6#1" --tx-in "$TX_6#2"               \
                               --tx-out "$WALLET_ADDRESS+2000000+1 $ROLES_CURRENCY.$MEDIATOR_ROLE" \
                               --tx-out "$WALLET_ADDRESS+2000000+1 $ROLES_CURRENCY.$BUYER_ROLE"    \
                               --tx-out "$WALLET_ADDRESS+2000000+1 $ROLES_CURRENCY.$SELLER_ROLE"   \
                               --change-address "$WALLET_ADDRESS"                                  \
                               --out-file /dev/null                                                \
                               --submit=600

TxId "e7597bea578ef473d07d99d6f59bd69121599306f222273245158ad364520008"


## Viewing Transactions on Cardano Explorer

| Transaction | Link |
|-------------|------|
| 1           | https://explorer.pioneers.testnet.marlowe-finance.io/en/transaction?id=bc4b7b7979da6f9f7f01b85ce3b44eacac838547944ef813a4059e19c75985be |
| 2           | https://explorer.pioneers.testnet.marlowe-finance.io/en/transaction?id=fd7dcb45cc607c21133087112f65ab5fbdb62e2d4359205aee2cb0f1ef2626b6 |
| 3           | https://explorer.pioneers.testnet.marlowe-finance.io/en/transaction?id=4c4462d48a2e4026944e65b8cfa61875039d4f6824b6239a76528a45194a52b6 |
| 4           | https://explorer.pioneers.testnet.marlowe-finance.io/en/transaction?id=53d91ef7534151278fecabb2f733c241ffbee0b488aa0dba4d9376a151170c7e |
| 5           | https://explorer.pioneers.testnet.marlowe-finance.io/en/transaction?id=4f597e8a4fcdc36da5326e82989005be03e1c0b4f7f0b30baa0cf31fb266c902 |
| 6           | https://explorer.pioneers.testnet.marlowe-finance.io/en/transaction?id=7ba827a37c5b4d59d8d8e5571089a169d8d5662f6b3a05e83c52c3e6852de1e5 |
| 7           | https://explorer.pioneers.testnet.marlowe-finance.io/en/transaction?id=86008a0ab907a413ab8b749c919f43cfbb36e605f165b3c7a55e1744b789c1ba |

## Progression of eUTxOs

![Boxes are UTxOs; ovals are transactions; ADA values are rounded to one decimal place.](diagrams/escrow-txs.png)

## Troubleshooting

*   See the previous lecture, "Running Marlowe Contracts without Blockchain Transactions", for hints at troubleshooting the semantic aspects of a Marlowe contract.
*   If a transaction fails phase two validation, a terse Marlowe error will be reported:
    *   `"B0"`: All accounts must have positive balances.
    *   `"B1"`: The value input from the script does not match that specified in its datum.
    *   If these errors occur during Plutus validation but not outside of it, then there likely is a clock inconsistency between local time and the node.
        *   `"E1" = TEAmbiguousTimeIntervalError`: The time interval for a transaction straddles the timeout of a `When` clause.
        *   `"E2" = TEApplyNoMatchError`: The attempted application of input to the contract was illegal.
        *   `"E3" = TEIntervalError InvalidInterval`: The start of the time interval is after its end.
        *   `"E4" = IntervalInPast`: The interval falls before the current time.
        *   `"E5" = TEUselessTransaction`: The attempted application of input does not change the state of the contract.
        *   `"E6" = TEHashMismatch`: An incorrect contract was provided for the merkleized continuation.
    *   `"H"`: The merkleized continuation for the contract is missing from the transaction.
    *   `"I0"`: The input datum was not provided to the script.
    *   `"I1"`: Inputting from two Marlowe scripts with the same address in the same transaction is forbidden.
    *   `"L1+"`: The datum or value at the script output does not match the contract's transition.
    *   `"O0"`: Outputing to two Marlowe scripts with the same address in the same transaction is forbidden.
    *   `"P"`: Insufficient value is paid to a public-key address.
    *   `"R"`: Insufficient value is paid in a role payout.
    *   `"R0"`: The slot/time validity range for the transaction is infinite or semi-infinite. The range should be finite/bounded.
    *   `"S"`: A required public-key signature was not present in the transaction.
    *   `"T"`: The role token authorizing the application of input is not present in the transaction.
*   Validation error codes not mentioned above result from Plutus or Plutus Apps functions. 

## Resources

*   Marlowe Debugging Cookbook: <<https://github.com/input-output-hk/marlowe-cardano/blob/mpp-cli-lectures/marlowe/debugging-cookbook.md>>.
*   Example contracts with `marlowe-cli`: <<https://github.com/input-output-hk/marlowe-cardano/blob/mpp-cli-lectures/marlowe-cli/examples/ReadMe.md>>.
*   Marlowe contract examples: <<https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-contracts>>.
*   Marlowe Playground: <<https://playground.marlowe.iohkdev.io/#/>>.
*   Marlowe Run
    *   On pioneers testnet: <<https://marlowe-run-marlowe-pioneers.plutus.aws.iohkdev.io/>>.
    *   On mock network: <<https://marlowe-finance.io/>>.
*   Cardano Docs for Marlowe: <<https://docs.cardano.org/marlowe/learn-about-marlowe>>.
*   Plutus Pioneers Program: <<https://github.com/input-output-hk/plutus-pioneer-program>>.
*   Plutus Community: <<https://plutus-community.readthedocs.io/en/latest/>>.
*   The Plutonomicon: <<https://github.com/Plutonomicon/plutonomicon/blob/main/README.md>>.

## Summary

*   Role tokens must be distributed to the parties involved in the transactions of a Marlowe contract.
*   The `marlowe-cli run execute` command submits Marlowe transactions to the blockchain.
*   The `marlowe-cli run withdraw` command submits a transaction to withdraw funds from Marlowe's payout script address.
*   Transactions can be viewed on the Cardano explore at <<https://explorer.pioneers.testnet.marlowe-finance.io/en>>.

## Other Lectures

Lectures on Marlowe CLI: <<https://github.com/input-output-hk/marlowe-cardano/blob/mpp-cli-lectures/marlowe-cli/lectures/ReadMe.md>>

*   Overview of Marlowe CLI
*   Installing Marlowe CLI and Associated Tools
*   Running Marlowe Contracts without Blockchain Transactions
*   ~~Running Marlowe Contacts on the Blockchain~~
*   Running Marlowe Contracts with the Plutus Application Backend (PAB)
*   Reference for Marlowe CLI Commands